In [8]:
import pandas as pd
import requests
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [10]:
# Load the datasets
movies_df = pd.read_csv('C:/Users/USER/Documents/CineIntellect_final/data/tmdb_5000_movies.csv')
credits_df = pd.read_csv('C:/Users/USER/Documents/CineIntellect_final/data/tmdb_5000_credits.csv')

In [11]:
# Preview the data
print("Movies DataFrame:")
display(movies_df.head())

Movies DataFrame:


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [12]:
print("Credits DataFrame:")
display(credits_df.head())

Credits DataFrame:


,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [13]:
# Merging the two datasets on 'id' and 'movie_id' columns
merged_df = pd.merge(movies_df, credits_df, left_on='id', right_on='movie_id')
print("Merged DataFrame:")
display(merged_df.head())


Merged DataFrame:


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,spoken_languages,status,tagline,title_x,vote_average,vote_count,movie_id,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [14]:
# Checking for missing values in the dataset
print("Missing values before cleaning:")
missing_data = merged_df.isnull().sum().sort_values(ascending=False)
display(missing_data[missing_data > 0])



Missing values before cleaning:


homepage        3091
tagline          844
overview           3
runtime            2
release_date       1
dtype: int64

In [17]:
print(merged_df.columns)

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title_x', 'vote_average',
       'vote_count', 'movie_id', 'title_y', 'cast', 'crew'],
      dtype='object')


In [19]:
# Check if the two title columns are the same
if merged_df['title_x'].equals(merged_df['title_y']):
    print("Both title columns are identical.")
    # Drop one of the title columns
    merged_df.drop(columns=['title_y'], inplace=True)
    # Rename the remaining title column for clarity
    merged_df.rename(columns={'title_x': 'title'}, inplace=True)
else:
    print("The title columns are different.")
    # Keeping the 'title_x' from the movies dataset
    merged_df.drop(columns=['title_y'], inplace=True)
    merged_df.rename(columns={'title_x': 'title'}, inplace=True)

Both title columns are identical.


In [20]:
print(merged_df.columns)

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')


In [21]:
# Handling Missing Data
# Drop rows with missing 'title' or 'overview', as these are important for recommendations
cleaned_df = merged_df.dropna(subset=['title', 'overview'])

In [22]:
# Fill other missing values with empty strings or zeros depending on the context
cleaned_df['cast'].fillna('', inplace=True)
cleaned_df['crew'].fillna('', inplace=True)

C:\Users\USER\AppData\Local\Temp\ipykernel_7892\689841436.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cleaned_df['cast'].fillna('', inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_7892\689841436.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['cast'].fillna('', inplace=True)
C:\Users\USER\AppData\Local\Temp\ipyk

In [23]:
# Confirming missing data after cleaning
print("Missing values after cleaning:")
cleaned_missing_data = cleaned_df.isnull().sum().sort_values(ascending=False)
display(cleaned_missing_data[cleaned_missing_data > 0])


Missing values after cleaning:


homepage        3088
tagline          841
release_date       1
dtype: int64

In [27]:
# Analyzing Genre Distribution
# Extract genres from the 'genres' column
cleaned_df.loc[:, 'genres_list'] = cleaned_df['genres'].apply(lambda x: [i['name'] for i in eval(x)] if pd.notna(x) else [])


In [28]:
# Flattening the list of genres for visualization
all_genres = [genre for sublist in cleaned_df['genres_list'] for genre in sublist]
genres_count = pd.Series(all_genres).value_counts()


In [32]:
# Plot the genre distribution
plt.figure(figsize=(12, 6))
sns.barplot(x=genres_count.index, y=genres_count.values, hue=genres_count.index, palette='viridis', legend=False)
plt.xticks(rotation=90)
plt.title("Genre Distribution in the Cleaned Dataset")
plt.ylabel("Number of Movies")
plt.tight_layout()

# Saving the plot to a file instead of showing it
plt.savefig("genre_distribution.png")

In [34]:
for col in cleaned_df.columns:
    if cleaned_df[col].apply(lambda x: isinstance(x, list)).any():
        print(f"Column '{col}' contains lists.")


Column 'genres_list' contains lists.


In [36]:
cleaned_df = cleaned_df.copy()

# Convert list columns to strings
for col in cleaned_df.columns:
    if cleaned_df[col].apply(lambda x: isinstance(x, list)).any():
        cleaned_df.loc[:, col] = cleaned_df[col].apply(lambda x: str(x) if isinstance(x, list) else x)



In [37]:
duplicates = cleaned_df.duplicated().sum()
print(f"Number of duplicate rows in the cleaned dataset: {duplicates}")


Number of duplicate rows in the cleaned dataset: 0


In [40]:
# Saving the cleaned dataset
cleaned_df.to_csv('cleaned_movies_with_credits.csv', index=False)

print("Data Cleaning Complete. Cleaned dataset saved as 'cleaned_movies_with_credits.csv'.")

Data Cleaning Complete. Cleaned dataset saved as 'cleaned_movies_with_credits.csv'.
